In [1]:
import sqlite3
import pandas as pd
import io

# 1. Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# --- Create Simulated ERP Tables ---
create_tables_sql = """
CREATE TABLE gl_entry (
    id INTEGER PRIMARY KEY,
    transaction_id TEXT NOT NULL,
    posting_date DATE NOT NULL,
    account_code TEXT NOT NULL,
    debit_amount REAL DEFAULT 0.00,
    credit_amount REAL DEFAULT 0.00
);
CREATE TABLE invoice (
    id INTEGER PRIMARY KEY,
    invoice_id TEXT UNIQUE NOT NULL,
    header_total REAL NOT NULL
);
CREATE TABLE invoice_item (
    id INTEGER PRIMARY KEY,
    invoice_id TEXT NOT NULL,
    amount REAL NOT NULL
);
"""
cursor.executescript(create_tables_sql)

# --- Populate Data (Including Errors) ---
insert_data_sql = """
-- INVOICES (INV-001: Header Mismatch, INV-002: Missing GL, INV-003: Duplicate GL, INV-004: Good)
INSERT INTO invoice (invoice_id, header_total) VALUES
('INV-001', 1000.00),
('INV-002', 500.00),
('INV-003', 1200.00),
('INV-004', 750.00);

-- INVOICE ITEMS (True total for INV-001 is 900.00)
INSERT INTO invoice_item (invoice_id, amount) VALUES
('INV-001', 500.00), ('INV-001', 400.00),
('INV-002', 500.00),
('INV-003', 1200.00),
('INV-004', 750.00);

-- GL ENTRIES (Accounts Receivable 1200 is the account we reconcile)
INSERT INTO gl_entry (transaction_id, posting_date, account_code, debit_amount, credit_amount) VALUES
('INV-001', '2025-10-15', '1200 - AR', 900.00, 0.00),
('INV-002', '2025-10-16', '4000 - Sales', 0.00, 500.00), -- INTENTIONALLY MISSING AR ENTRY
('INV-003', '2025-10-17', '1200 - AR', 1200.00, 0.00),
('INV-003', '2025-10-17', '1200 - AR', 1200.00, 0.00), -- DUPLICATE ENTRY
('INV-004', '2025-10-18', '1200 - AR', 750.00, 0.00);
"""
cursor.executescript(insert_data_sql)
conn.commit()
print("Database and Sample Data Setup Complete.")

Database and Sample Data Setup Complete.


In [2]:
# --- SQL Extraction: Pull all relevant data into one table ---
reconciliation_query = """
WITH GL_AR_Data AS (
    -- Group GL entries to handle duplicates and get the total AR posting
    SELECT
        transaction_id,
        SUM(debit_amount - credit_amount) AS gl_amount
    FROM
        gl_entry
    WHERE
        account_code = '1200 - AR' -- Focus only on Accounts Receivable
    GROUP BY transaction_id
),
Invoice_Line_Totals AS (
    -- Calculate the TRUE total from the line items
    SELECT
        invoice_id,
        SUM(amount) AS line_item_total
    FROM
        invoice_item
    GROUP BY invoice_id
)
-- Combine everything
SELECT
    i.invoice_id,
    i.header_total,           -- Invoice Header Total (Source 1)
    ilt.line_item_total,      -- True Invoice Total (Source 2)
    gld.gl_amount             -- Total Posted to GL (Source 3)
FROM
    invoice i
LEFT JOIN
    Invoice_Line_Totals ilt ON i.invoice_id = ilt.invoice_id
LEFT JOIN
    GL_AR_Data gld ON i.invoice_id = gld.transaction_id;
"""

# Execute SQL and load into Pandas DataFrame
df = pd.read_sql(reconciliation_query, conn)
conn.close()

# --- Pandas Reconciliation Logic ---

df['Discrepancy_Type'] = ''

# 1. Check: Header Total vs. Line Item Total Mismatch
df.loc[
    abs(df['header_total'] - df['line_item_total']) > 0.01,
    'Discrepancy_Type'
] += 'Header/Line Mismatch; '

# 2. Check: Missing Accounts Receivable GL Entry (gl_amount is NaN/null)
df.loc[
    df['gl_amount'].isna(),
    'Discrepancy_Type'
] += 'Missing AR GL Entry; '

# 3. Check: GL Posted Amount vs. True Line Item Total Mismatch (Includes Duplicates)
df.loc[
    df['gl_amount'].notna() & (abs(df['gl_amount'] - df['line_item_total']) > 0.01),
    'Discrepancy_Type'
] += 'GL/True Total Mismatch or Duplication; '

# Filter and clean the final report
discrepancy_report = df[df['Discrepancy_Type'] != ''].copy()
discrepancy_report['Discrepancy_Type'] = discrepancy_report['Discrepancy_Type'].str.rstrip('; ')
discrepancy_report['Variance'] = discrepancy_report['gl_amount'].fillna(0) - discrepancy_report['line_item_total']

# --- Export to Excel ---
output_excel_path = 'Automated_Reconciliation_Report.xlsx'
discrepancy_report.to_excel(output_excel_path, index=False, sheet_name='Discrepancy_Raw')

print("\n--- Discrepancy Report Generated ---")
print(f"File '{output_excel_path}' created in Colab files.")


--- Discrepancy Report Generated ---
File 'Automated_Reconciliation_Report.xlsx' created in Colab files.
